In [1]:
# pip install NorenRestApiPy-0.0.16-py2.py3-none-any.whl

In [ ]:
import pandas as pd
import pytz, time
import pyotp
import requests
import json
from urllib.parse import parse_qs,urlparse
import traceback
import hashlib

## Credentials

In [ ]:
APIKEY=''
secretKey = ''
totp_key=''
password = ''
userid = ''
headerJson =  {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36", "Referer":"https://auth.flattrade.in/"}



In [ ]:
ses = requests.Session()
sesUrl = 'https://authapi.flattrade.in/auth/session'
passwordEncrpted =  hashlib.sha256(password.encode()).hexdigest()
ses = requests.Session()

res_pin = ses.post(sesUrl,headers=headerJson)
sid = res_pin.text
print(f'sid {sid}')
url2 = 'https://authapi.flattrade.in/ftauth'
payload = {"UserName":userid,"Password":passwordEncrpted,"PAN_DOB":pyotp.TOTP(totp_key).now(),"App":"","ClientID":"","Key":"","APIKey":APIKEY,"Sid":sid,
          "Override":"Y","Source":"AUTHPAGE"}
res2 = ses.post(url2, json=payload)
reqcodeRes = res2.json()
print(reqcodeRes)
parsed = urlparse(reqcodeRes['RedirectURL'])  
reqCode = parse_qs(parsed.query)['code'][0]
api_secret =APIKEY+ reqCode + secretKey 
api_secret =  hashlib.sha256(api_secret.encode()).hexdigest()
payload = {"api_key":APIKEY, "request_code":reqCode, "api_secret":api_secret}
url3 = 'https://authapi.flattrade.in/trade/apitoken'  
res3 = ses.post(url3, json=payload)
print(res3.json())
token = res3.json()['token']
token

## Create Broker object 

In [ ]:
from NorenRestApiPy.NorenApi import  NorenApi

class FlatTradeApiPy(NorenApi):
    def __init__(self):
          NorenApi.__init__(self, host='https://piconnect.flattrade.in/PiConnectTP/', websocket='wss://piconnect.flattrade.in/PiConnectWSTp/', eodhost='https://web.flattrade.in/chartApi/getdata/')
        
api = FlatTradeApiPy()  
ret = api.set_session(userid= userid, password = password, usertoken= token)

api.get_limits()

In [ ]:
api.get_quotes(exchange='BSE', token='1')